# How to use pythreejs to plot a superellipsoid

A superellipsoid is given by a parametric function and the equation is very similar to an ellipse equation. We only have different exponents which give us different shapes. For more informations: https://en.wikipedia.org/wiki/Superellipsoid.

The idea of this example is to construct the mesh of the square $[0, 1]\times[0,1]$ and to do a projection of these points on the superillipse which is the 2D shape and then to do a spherical product to have the 3D shape.

In [1]:
from pythreejs import *
from ipywidgets import IntSlider, FloatSlider, HBox, VBox
from IPython.display import display
import numpy as np

In [2]:
def superellipsoid(n, rx, ry, rz, m1, m2):
    """
    superellipsoid formula with the spherical product of two superellipse
    and update of the global coords array
    
    Parameters
    ----------
    rx : the radius in the x direction 
    ry : the radius in the y direction 
    rz : the radius in the z direction 
    m1 : the exponent of the first superellipse
    m2 : the exponent of the second superellipse
    """
    
    x_box = np.concatenate((np.linspace(-1, 1., n), np.ones(n-2), np.linspace(1, -1., n), -np.ones(n-2)))
    y_box = np.concatenate((-np.ones(n-1), np.linspace(-1, 1., n), np.ones(n-2), np.linspace(1, -1., n-1, endpoint=False)))
    nx_box = x_box.size

    coords = np.empty((nx_box**2, 3), dtype=np.float32)
    
    def superellipse(rx, ry, m):
        """
        superellipse formula with the projection of the unit square

        Parameters
        ----------
        rx : the radius in the x direction 
        ry : the radius in the y direction 
        m : the exponent of the superellipse

        Output
        ------
        the coordinates of the superellipse
        """
        return x_box*rx*(1. - .5*np.abs(y_box)**(2./m))**(m/2.),  y_box*ry*(1. - .5*np.abs(x_box)**(2./m))**(m/2.)

    gx, gy = superellipse(1, 1, m2)
    hx, hy = superellipse(1, 1, m1)
    
    coords[:, 0] = rx*(gx[np.newaxis, :]*hx[:, np.newaxis]).flatten()
    coords[:, 1] = ry*(gx[np.newaxis, :]*hy[:, np.newaxis]).flatten()
    coords[:, 2] = rz*(gy[np.newaxis, :]*np.ones(hx.size)[:, np.newaxis]).flatten()
    
    return coords

In [3]:
# superellipsoid parameters
n = 20
rx = ry = rz = 1.
m1 = m2 = 1.

coords = superellipsoid(n, rx, ry, rz, m1, m2)

In [4]:
view_width = 600
view_height = 400

vertices = BufferAttribute(
    array=coords,
    normalized=False)

meshGeometry = BufferGeometry(
    attributes={
        'position': vertices,
    }
)

In [6]:
texture = ImageTexture('disc.png')

In [7]:
pointsMaterial = PointsMaterial(
    color='#80bfff',
    map=texture,
    size=.1,
    alphaTest=0.5,
)

In [8]:
points = Points(
    geometry=meshGeometry,
    material=pointsMaterial    
)

In [9]:
camera = PerspectiveCamera(
    position=[2, 2, 1],
    aspect=view_width / view_height,
    near=1,
    far=1000,
    children=[PointLight("0xffffff", 0 )])

In [10]:
controls = OrbitControls(
    controlling=camera,
)

In [11]:
ambient_light = AmbientLight('0x222222')

In [12]:
scene = Scene(
    children=[camera, ambient_light, AxesHelper(1), points],
    background = 'black',
)

In [13]:
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[controls],
    width=view_width, 
    height=view_height)

In [14]:
spin_track = NumberKeyframeTrack(name='.rotation[y]', times=[0, 1000], values=[0, 100])
spin_clip = AnimationClip(tracks=[spin_track])
spin_action = AnimationAction(AnimationMixer(points), spin_clip, points)

In [15]:
n_slider, m1_slider, m2_slider = (
    IntSlider(description='n', min=5, max=50, step=1, value=n,
              continuous_update=False, orientation='vertical'),
    FloatSlider(description='m1', min=0.01, max=4.0, step=0.01, value=m1,
                continuous_update=False, orientation='vertical'),
    FloatSlider(description='m2', min=0.01, max=4.0, step=0.01, value=m2,
                continuous_update=False, orientation='vertical')
)

In [16]:
rx_slider, ry_slider, rz_slider = (FloatSlider(description='rx', min=0.01, max=10.0, step=0.1, value=rx, 
                                               continuous_update=False, orientation='horizontal'),
                                   FloatSlider(description='ry', min=0.01, max=10.0, step=0.1, value=ry, 
                                               continuous_update=False, orientation='horizontal'),
                                   FloatSlider(description='rz', min=0.01, max=10.0, step=0.1, value=rz, 
                                               continuous_update=False, orientation='horizontal'))

In [17]:
def update(change):
    coords = superellipsoid(n_slider.value, rx_slider.value, ry_slider.value, rz_slider.value, 
                   m1_slider.value, m2_slider.value)
    vertices.array = coords
    
n_slider.observe(update, names=['value'])
m1_slider.observe(update, names=['value'])
m2_slider.observe(update, names=['value'])
rx_slider.observe(update, names=['value'])
ry_slider.observe(update, names=['value'])
rz_slider.observe(update, names=['value'])

In [18]:
VBox([HBox([renderer, 
            VBox([HBox([m1_slider, m2_slider, n_slider]), spin_action])
           ]),
      rx_slider, 
      ry_slider,
      rz_slider])

In [19]:
renderer._width = 1000

In [20]:
renderer._height = 800